In [1]:
# import our modules
import re
import requests
import unicodedata
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
# set our root url
root_url = "http://www.spotrac.com/nfl"

In [3]:
# request page using a "GET" request
r = requests.get(root_url)
# actually get the text of the page (in this case, HTML!)
page_text = r.text
# store the HTML in a BS object; we'll use the "lxml" parsing system
soup = bs(page_text, "lxml")
# we can actually see our web page stored in the BS object using one of the BS methods
# for example, here's a sample 400 characters...
print soup.prettify()[7600:8000]

ol-xs-4 text-right">
       <a href="#" id="nav-expander" onclick="return false;">
        <span class="fa fa-bars" style="z-index:1000000;position:relative;">
        </span>
       </a>
      </div>
      <!-- end:col -->
     </div>
     <!-- end:row -->
     <!-- start:sidr -->
     <div id="sidr">
      <form action="https://www.spotrac.com/redirect/search/" id="search-form-mobile" method="po


In [4]:
links = soup.find_all('a')
temp = []
for link in links:
    if (link.text[2:] == "Positional Spending"):
        temp.append(link.get('href'))
links=temp
links.remove("http://www.spotrac.com/nfl/los-angeles-chargers/positional/")
links.insert(26,"http://www.spotrac.com/nfl/san-diego-chargers/positional/")
print(links)
teams=[]
for l in links:
    team=l[::-1][12:]
    team=team[:team.find("/")][::-1]
    team=team.replace("-", " ")
    teams.append(team.title())
teams[27]="San Francisco 49ers"
newlinks=links[:]
newteams=teams[:]
oldrams="http://www.spotrac.com/nfl/st.-louis-rams/positional/"
oldlinks=links[:]
oldlinks.remove("http://www.spotrac.com/nfl/los-angeles-rams/positional/")
oldlinks.insert(28, oldrams)
oldramsname="St. Louis Rams"
oldteams=teams[:]
oldteams.remove("Los Angeles Rams")
oldteams.insert(28, oldramsname)
print(oldteams)

['http://www.spotrac.com/nfl/arizona-cardinals/positional/', 'http://www.spotrac.com/nfl/atlanta-falcons/positional/', 'http://www.spotrac.com/nfl/baltimore-ravens/positional/', 'http://www.spotrac.com/nfl/buffalo-bills/positional/', 'http://www.spotrac.com/nfl/carolina-panthers/positional/', 'http://www.spotrac.com/nfl/chicago-bears/positional/', 'http://www.spotrac.com/nfl/cincinnati-bengals/positional/', 'http://www.spotrac.com/nfl/cleveland-browns/positional/', 'http://www.spotrac.com/nfl/dallas-cowboys/positional/', 'http://www.spotrac.com/nfl/denver-broncos/positional/', 'http://www.spotrac.com/nfl/detroit-lions/positional/', 'http://www.spotrac.com/nfl/green-bay-packers/positional/', 'http://www.spotrac.com/nfl/houston-texans/positional/', 'http://www.spotrac.com/nfl/indianapolis-colts/positional/', 'http://www.spotrac.com/nfl/jacksonville-jaguars/positional/', 'http://www.spotrac.com/nfl/kansas-city-chiefs/positional/', 'http://www.spotrac.com/nfl/los-angeles-rams/positional/',

In [6]:
years = range(2013, 2017)
database = np.zeros(shape=(3584,7),dtype="a30")
counter = 0
for year in years:
    if (year < 2016):
        links = oldlinks
        teams = oldteams
    else:
        links = newlinks
        teams = newteams
    for i in range(len(links)):
        r = requests.get(links[i] + str(year) + "/full-cap/")
        soup = bs(r.text, "lxml")
        data = soup.find_all('td')
        for j in range(len(data)):
            if j % 5 != 0:
                continue
            else:
                database[counter]=[str(year), newteams[i], str(data[j].text), str(data[j+1].text), str(data[j+2].text), str(data[j+3].text), str(data[j+4].text)]
                counter += 1
np.savetxt("nfldata.txt",database,delimiter="\t",fmt="%s")
        

In [52]:
# Combine data into a single row per team/year
df = pd.read_csv("nfldata.csv",delimiter=",")
final = pd.read_csv("NFLRecords.csv", delimiter=",")

In [53]:
allPositions=[]
for i in range(1024, 1052):
    pos = df.iloc[i]["Position"]
    allPositions.append(pos)
    print(pos)
    final[pos + " Spending"] = pd.Series(np.random.randn(len(final.index)), index=final.index)

Offense
Defensive Line
Secondary
Offensive Line
Cornerback
Defensive Tackle
Defensive End
Linebacker
Guard
Quarterback
Wide Receiver
Tight End
Inside Linebacker
Tackle
Safety
Outside Linebacker
Left Tackle
Free Safety
Special Teams
Running Back
Punter
Strong Safety
Kicker
Center
Right Tackle
Long Snapper
Fullback
Defense


In [54]:
final.head()

,Year,Team,Wins,Losses,Ties,Points For,Points Against,Sos,Offense Spending,Defensive Line Spending,...,Special Teams Spending,Running Back Spending,Punter Spending,Strong Safety Spending,Kicker Spending,Center Spending,Right Tackle Spending,Long Snapper Spending,Fullback Spending,Defense Spending
0,2013,Arizona Cardinals,10,6,0,379,324,3.0,-0.393142,1.146481,...,-1.281490,0.997447,-1.679102,0.315049,-0.308717,1.040150,1.888424,0.978406,0.852354,-0.085231
1,2013,Atlanta Falcons,4,12,0,353,443,2.9,0.023561,1.144344,...,0.351610,-1.374967,-0.067078,-1.256733,-0.917782,0.106773,0.340172,-0.178541,0.557086,0.959452
2,2013,Baltimore Ravens,8,8,0,320,352,-1.5,1.634853,0.023265,...,0.995768,-1.044803,0.934788,-0.130789,-0.144736,-1.065336,0.792538,1.048102,0.236449,0.522724
3,2013,Buffalo Bills,6,10,0,339,388,-0.1,-1.266526,-0.793290,...,2.471545,1.060566,-0.726084,0.353341,-0.384712,-1.851124,-0.897414,1.317056,1.638967,1.142741
4,2013,Carolina Panthers,12,4,0,366,241,1.4,-0.147095,-1.060470,...,-1.339188,0.678241,1.352090,-1.719133,-0.224588,-0.922461,-0.035388,-0.493501,-0.299823,-0.244274


In [55]:
year="2013"
team="Arizona Cardinals"
filledPositions = []
counter = 0
for i in range(len(df.index)):
    if str(df.iloc[i]["Year"]) != year or df.iloc[i]["Team"] != team:
        missing = list(set(allPositions) - set(filledPositions))
        for pos in missing:
            final.set_value(counter, pos + " Spending", None)
        counter += 1
        filledPositions= []
        year = str(df.iloc[i]["Year"])
        team = df.iloc[i]["Team"]
    pos = df.iloc[i]["Position"]
    spending = df.iloc[i]["% of Cap"][:-1]
    final.set_value(counter, pos + " Spending", spending)
    filledPositions.append(pos)

In [49]:
final.set_value(0, "Offense Spending", 10)

,Year,Team,Wins,Losses,Ties,Points For,Points Against,Sos,Offense Spending,Defensive Line Spending,...,Running Back Spending,Punter Spending,Strong Safety Spending,Kicker Spending,Center Spending,Right Tackle Spending,Long Snapper Spending,Fullback Spending,Defense Spending,Kick Returner Spending
0,2013.0,Arizona Cardinals,10.0,6.0,0.0,379.0,324.0,3.0,10,None,...,None,None,None,None,None,None,None,None,None,NaN
1,2013.0,Atlanta Falcons,4.0,12.0,0.0,353.0,443.0,2.9,None,None,...,None,None,None,None,None,None,None,None,None,NaN
2,2013.0,Baltimore Ravens,8.0,8.0,0.0,320.0,352.0,-1.5,None,None,...,None,None,None,None,None,None,None,None,None,NaN
3,2013.0,Buffalo Bills,6.0,10.0,0.0,339.0,388.0,-0.1,None,None,...,None,None,None,None,None,None,None,None,None,NaN
4,2013.0,Carolina Panthers,12.0,4.0,0.0,366.0,241.0,1.4,None,None,...,None,None,None,None,None,None,None,None,None,NaN
5,2013.0,Chicago Bears,8.0,8.0,0.0,445.0,478.0,-2.1,None,None,...,None,None,None,None,None,None,None,None,None,NaN
6,2013.0,Cincinnati Bengals,11.0,5.0,0.0,430.0,305.0,-2.5,None,None,...,None,None,None,None,None,None,None,None,None,NaN
7,2013.0,Cleveland Browns,4.0,12.0,0.0,308.0,406.0,-1.6,None,None,...,None,None,None,None,None,None,None,None,None,NaN
8,2013.0,Dallas Cowboys,8.0,8.0,0.0,439.0,432.0,-1.1,None,None,...,None,None,None,None,None,None,None,None,None,NaN
9,2013.0,Denver Broncos,13.0,3.0,0.0,606.0,399.0,-1.6,None,None,...,None,None,None,None,None,None,None,None,None,0.36


In [57]:
del final["Kick Returner Spending"]
final.head()

,Year,Team,Wins,Losses,Ties,Points For,Points Against,Sos,Offense Spending,Defensive Line Spending,...,Special Teams Spending,Running Back Spending,Punter Spending,Strong Safety Spending,Kicker Spending,Center Spending,Right Tackle Spending,Long Snapper Spending,Fullback Spending,Defense Spending
0,2013,Arizona Cardinals,10,6,0,379,324,3.0,41.68,18.24,...,NaN,4.42,1.31,0.34,1.71,3.66,2.59,0.91,NaN,38.98
1,2013,Atlanta Falcons,4,12,0,353,443,2.9,46.92,9.24,...,NaN,4.86,0.47,3.53,2.42,1.28,1.93,0.39,0.72,35.49
2,2013,Baltimore Ravens,8,8,0,320,352,-1.5,36.55,16.51,...,NaN,5.06,1.96,2.47,0.38,0.85,1.22,0.59,1.49,46.82
3,2013,Buffalo Bills,6,10,0,339,388,-0.1,36.20,25.19,...,NaN,6.65,0.34,1.83,0.79,5.17,2.98,0.92,0.61,45.71
4,2013,Carolina Panthers,12,4,0,366,241,1.4,45.97,15.10,...,NaN,7.63,0.45,3.66,0.65,6.54,1.07,0.78,2.09,35.17


In [59]:
final.to_csv("nflfinal.csv",sep=",", index=False)